# <img style="float: left; padding-right: 10px; width: 150px" src="https://sc.acamica.com/icons/1j7w9h/social-300x300.png">  Acámica DS-COR3 - Ejercicio de Redes Neuronales

### Junio 2019

<hr style="height:2pt">

## Descripción

El objetivo de este notebook es ejercitar el ajuste de una Red Neuronal básica sobre un conjunto de datos de dígitos manuscritos de MNIST. En este caso se utilizará Keras como librería para modelar un perceptrón multicapa (MLP), que además provee el dataset a ser utilizado. El notebook está basado en un [ejemplo provisto por Keras](https://github.com/keras-team/keras/blob/master/examples/mnist_mlp.py), por lo cual el ejercicio es seguir paso a paso el desarrollo del mismo y probar ciertos cambios propuestos por los items marcados como **\*\*PREGUNTAS\*\***, así como documentar el efecto de aplicar esos cambios. 

**Recursos**:  
- https://github.com/keras-team/keras/blob/master/examples/mnist_mlp.py
- https://medium.com/@mjbhobe/mnist-digits-classification-with-keras-ed6c2374bd0e
- https://www.kaggle.com/adityaecdrid/mnist-with-keras-for-beginners-99457
- https://towardsdatascience.com/deep-learning-tips-and-tricks-1ef708ec5f53

In [1]:
# Dependencias
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.utils import to_categorical

Using TensorFlow backend.


## Data preparation

In [2]:
# Cargar datasets de MNIST, en conjuntos de train y test
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Flatten de matrices de 28x28 a vectores de 784 elementos
x_train = x_train.reshape(len(x_train), 784)
x_test = x_test.reshape(len(x_test), 784)

# Casting de int a float
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Min Max scaling: Rango [0, 255] -> [0, 1]
# **PREGUNTA**: Qué efecto tiene en el entrenamiento no hacerlo? 
x_train /= 255
x_test /= 255

# Por último, separar un conj de validación para monitoreo
# Dado que ya está shuffled, se puede tomar los primeros N
x_valid = x_train[-10000:, :]
y_valid = y_train[-10000:]
x_train = x_train[:-10000, :]
y_train = y_train[:-10000]

print(x_train.shape[0], 'train samples')
print(x_valid.shape[0], 'valid samples')
print(x_test.shape[0], 'test samples')

50000 train samples
10000 valid samples
10000 test samples


In [3]:
num_classes = 10

# Convertir etiquetas: vector de 10 categorías en matriz binaria de 10 elementos
# De esta forma, ahora el modelo provee como salida una probabilidad por cada clase
y_train = to_categorical(y_train, num_classes)
y_valid = to_categorical(y_valid, num_classes)
y_test = to_categorical(y_test, num_classes)

## Modeling

In [4]:
# Primero se define la ARQUITECTURA
# **PREGUNTAS:**
# 1. Qué pasa si aumento / reduzco la cantidad de neuronas en la primer capa?
# 2. Qué efecto tiene agregar 2 capas de 512 neuronas?
# 3. Qué ocurre si elimino las capas de DropOut?
# 4. Qué efecto tiene cambiar las activaciones de 'relu' a 'tanh'
# 5. Probar agregar una capa de BatchNormalization luego de la primer capa oculta. Qué ocurre y por qué?

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_______________

In [5]:
# Luego definimos la OPTIMIZACIÓN
# **PREGUNTAS:**
# 1. Qué efecto tiene usar un optimizador de tasa fija como SGD
# 2. Qué ocurre si se aumenta la tasa de aprendizaje (parámetro 'lr')
# 3. Qué pasa si uso un batch_size de 10? y uno de 500?
# 4. Analizar la evolución del loss en train y valid, en términos de las épocas (probar hasta 100) 

batch_size = 128
epochs = 20

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [6]:
# Ajustar el modelo!
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_valid, y_valid))


Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 4s 76us/step - loss: 0.2626 - acc: 0.9191 - val_loss: 0.1316 - val_acc: 0.9608
Epoch 2/20
50000/50000 [==============================] - 4s 71us/step - loss: 0.1089 - acc: 0.9662 - val_loss: 0.0935 - val_acc: 0.9721
Epoch 3/20
50000/50000 [==============================] - 4s 72us/step - loss: 0.0788 - acc: 0.9764 - val_loss: 0.1041 - val_acc: 0.9702
Epoch 4/20
50000/50000 [==============================] - 4s 72us/step - loss: 0.0611 - acc: 0.9820 - val_loss: 0.0796 - val_acc: 0.9776
Epoch 5/20
50000/50000 [==============================] - 4s 72us/step - loss: 0.0534 - acc: 0.9838 - val_loss: 0.0846 - val_acc: 0.9783
Epoch 6/20
50000/50000 [==============================] - 4s 72us/step - loss: 0.0449 - acc: 0.9862 - val_loss: 0.0946 - val_acc: 0.9787
Epoch 7/20
50000/50000 [==============================] - 4s 72us/step - loss: 0

In [7]:
# Finalmente evaluamos los resultados en test
# **PREGUNTAS**
# 1. Puede haber inconveniente de utilizar "accuracy" para interpretar el desempeño? 
#    (Pensar en cant de clases, desbalance en los datos, etc)
# 2. Calcular precision, recall y f1-score sobre test e interpretar resultados
# 3. Cómo se compara el loss en test con el de train? Qué podría indicar?

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.12511385221116994
Test accuracy: 0.9824
